In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [3]:
import mdtraj, nglview

In [3]:
rcParams.update({'font.size': 16})

In [6]:
# Switch AUTORELOAD ON. Disable this when in production mode!
%load_ext autoreload
%autoreload 2

In [4]:
from deep_boltzmann.networks.invertible import EnergyInvNet, invnet
from deep_boltzmann.networks.plot import test_xz_projection, test_generate_x
from deep_boltzmann.util import count_transitions
from deep_boltzmann.sampling.analysis import free_energy_bootstrap, mean_finite, std_finite
from deep_boltzmann.networks.training import MLTrainer, FlexibleTrainer
from deep_boltzmann.util import save_obj, load_obj

BPTI
-----

In [1]:
import numpy as np
from matplotlib import rcParams
import keras
import tensorflow as tf

from keras.utils.vis_utils import model_to_dot

from deep_boltzmann.networks.invertible import invnet
from deep_boltzmann.models.openmm import OpenMMEnergy
from deep_boltzmann import openmmutils
from deep_boltzmann.networks.invertible import EnergyInvNet
import mdtraj as md
from simtk import openmm, unit

Using TensorFlow backend.


In [2]:
# setup BPTI
INTEGRATOR_ARGS = (300*unit.kelvin,
                   1.0/unit.picoseconds,
                    2.0*unit.femtoseconds)

data_sets=[]
pdb = openmm.app.PDBFile('./bpti/bpti.pdb')
forcefield = openmm.app.ForceField('amber99sbildn.xml', 'amber99_obc.xml')

system = forcefield.createSystem(pdb.topology,
    nonbondedMethod=openmm.app.CutoffNonPeriodic, nonbondedCutoff=1.0*unit.nanometers,
    constraints=None, rigidWater=True)
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds,
    2.0*unit.femtoseconds)

In [3]:
simulation = openmm.app.Simulation(pdb.topology, system, integrator)

In [4]:
bpti_omm_energy = OpenMMEnergy(system,
                               openmm.LangevinIntegrator,
                               unit.nanometers,
                               n_atoms=md.Topology().from_openmm(simulation.topology).n_atoms,
                               openmm_integrator_args=INTEGRATOR_ARGS
                              )

mdtraj_topology = md.Topology().from_openmm(pdb.topology)

cartesian = ['CA', 'C', 'N']
cart = mdtraj_topology.select(' '.join(["name " + s for s in cartesian]))
from deep_boltzmann.models.proteins import mdtraj2Z
Z_ = np.array(mdtraj2Z(mdtraj_topology))

In [5]:
X0 = pdb.getPositions(asNumpy=True).ravel().reshape(1,-1).astype('float32')

In [13]:
BG = invnet(X0.shape[1], 'IRRRRRRRR', bpti_omm_energy, nl_layers=4, nl_hidden=128, 
            nl_activation='relu', nl_activation_scale='tanh',
           ic=Z_, ic_cart=cart, ic_cart_norm=X0)

II<RRRRRRRR>
I 2676 0 0


/opt/miniconda3/envs/BG/lib/python3.7/site-packages/Deep_Boltzmann_Package-0.1-py3.7.egg/deep_boltzmann/networks/invertible_coordinate_transforms.py:22: RuntimeWarning: invalid value encountered in true_divide
/opt/miniconda3/envs/BG/lib/python3.7/site-packages/Deep_Boltzmann_Package-0.1-py3.7.egg/deep_boltzmann/networks/invertible_coordinate_transforms.py:761: RuntimeWarning: invalid value encountered in log


I 2670 0 6
< 2664 0 6
R 1332 1332 6
R 1332 1332 6
R 1332 1332 6
R 1332 1332 6
R 1332 1332 6
R 1332 1332 6
R 1332 1332 6
R 1332 1332 6
> 1332 1332 6


In [14]:
loss_bg_trainML, loss_bg_valML = BG.train_ML(X0, epochs=15, batch_size=1000, lr=0.0001,
                                             std=1.0, verbose=1, return_test_energies=False)

Epoch 0/15 loss nan 
Epoch 1/15 loss nan 
Epoch 2/15 loss nan 
Epoch 3/15 loss nan 
Epoch 4/15 loss nan 
Epoch 5/15 loss nan 
Epoch 6/15 loss nan 
Epoch 7/15 loss nan 
Epoch 8/15 loss nan 
Epoch 9/15 loss nan 
Epoch 10/15 loss nan 
Epoch 11/15 loss nan 
Epoch 12/15 loss nan 
Epoch 13/15 loss nan 
Epoch 14/15 loss nan 


In [12]:
BG.energy_model.energy(X0)

array([-3401.8135], dtype=float32)

In [19]:
hasattr(BG.layers[0], 'log_det_zx')

True